<a href="https://colab.research.google.com/github/abhishekshakya/BPSO-and-ANN-for-sofware-fault-predicition/blob/master/internPSO_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
#https://ieeexplore.ieee.org/abstract/document/8011255
#from SVM https://www.youtube.com/watch?v=_PwhiWxHK8o

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from scipy.io import arff

In [2]:
df_pc1 = pd.read_csv('/content/drive/My Drive/dataset/pc1.csv')
df_kc1 = pd.read_csv('/content/drive/My Drive/dataset/kc1.csv')
df_kc2 = pd.read_csv('/content/drive/My Drive/dataset/kc2.csv')
df_jm1 = pd.read_csv('/content/drive/My Drive/dataset/jm1.csv')   

##Data cleaning

#####For kc2, here y feature is in object type

In [3]:
#for kc2 here change output feature from object -> bool
p = df_kc2['defects']
map = {'yes':True, 'no':False}
df_kc2['defects'] = p.map(map)


In [54]:
def normalize(x):
  x = x.astype(float)
  min = np.min(x)
  max = np.max(x)
  return (x - min)/(max-min)

####For jm1, we have '?' randomly distributed so we need to remove them also

In [5]:
d1,d2,d3,d4,d5 = df_jm1.iloc[:,16], df_jm1.iloc[:,17], df_jm1.iloc[:,18], df_jm1.iloc[:,19], df_jm1.iloc[:,20]

for i in range(len(d1)):
  if (str)(d1[i]) == '?' or (str)(d2[i]) == '?' or (str)(d3[i]) == '?' or (str)(d4[i]) == '?' or (str)(d5[i]) == '?':
    df_jm1.drop(i,inplace=True)
    i -= 1


In [ ]:
# print(x_real)
# print(y_real)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [7]:
class ANN(nn.Module):
  def __init__(self,features, nodes1, nodes2):
    super().__init__()
    self.l1 = nn.Linear(features, nodes1)
    self.l2 = nn.Linear(nodes1, nodes2)
    self.l3 = nn.Linear(nodes2 ,1)

  def forward(self,x):
    x = F.relu(self.l1(x))
    x = F.relu(self.l2(x))
    x = (self.l3(x))
    return torch.sigmoid((x))

class network:
  def __init__(self, nodes1, nodes2, dim, r):
    """nodes = number of neural nodes in first layer
       dim = number of features
       r = learning rate
      """
    self.ANN = ANN(dim, nodes1, nodes2)
    self.optim = optim.Adam(self.ANN.parameters(), lr=r)
    self.criterion = nn.BCELoss()

  def fit(self, x, y, iterations):
    ANN = self.ANN
    optimizer = self.optim
    criterion = self.criterion
    x = torch.from_numpy(x).float()
    y = torch.from_numpy(y).float()
    # y = y.reshape(y.shape[0],1)

    for i in (range(iterations)):

      optimizer.zero_grad()
    
      y_pred = ANN(x)
      y_pred = y_pred.squeeze()
      loss = criterion(y_pred, y)

      loss.backward()
      optimizer.step()
      
      # if i%((int)(iterations*0.05)) == 0:
      #   print(f"epoch {i}/{iterations}| loss=>{loss.item()}")
  
  def predict(self, x):
    ANN = self.ANN
    x = torch.from_numpy(x).float()
    y_ = ANN(x)
    y_ = y_.reshape(y_.shape[0])
    y_res = []
    for i in range(len(y_)):
      if y_[i] >= 0.5 :
        y_res.append(1)
      else:
        y_res.append(0)
    return y_res

In [32]:
class PSO:
  def __init__(self,particles, dimensions, lb, ub, itrations, c1, c2, w, xr, yr, node1, node2, lr, epochs):
    self.node1 = node1
    self.node2 = node2
    self.lr = lr
    self.epochs = epochs
    self.xr = xr
    self.yr = yr
    self.lb = lb
    self.ub = ub
    self.p = particles
    self.d = dimensions
    self.n = itrations
    self.c1 = c1
    self.c2 = c2
    self.w = w
    self.fitness_value = np.zeros(particles)#p
    self.X = np.random.randint(0,2,(particles,dimensions))#pxd
    self.V = np.random.randint(0,2,(particles,dimensions))#pxd
    self.pbest = self.X#pxd
    self.gbest = np.zeros(dimensions)#d
    #---initialiastion-------------------------
    max = 0
    for i in range(self.p):
      fit = self.fitness(self.X[i])
      self.fitness_value[i] = fit
      if max < fit:
        max = fit
        self.gbest = self.X[i]
    #-------------------------------------------
    self.maxg = max
    self.run()

  def sigmoid(self,x):
    return 1/(1+np.exp(-x))
  
  def count(self,x):
    c = np.sum(x)
    return c

  def fitness(self,x):
    x_train = []
    y_train = self.yr

    for i in range(self.d):
      if x[i] == 1:
        x_train.append(self.xr[:,i])

    x_train = np.array(x_train).reshape((self.xr.shape[0],-1))

    if x_train.shape[1] == 0:#no feature selected then return 0 acc
      return 0

    net = network(self.node1,self.node2, x_train.shape[1], self.lr)
    net.fit(x_train, y_train, self.epochs)
    ans = net.predict(x_train)

    acc = ((ans==y_train).sum()*1.0)/len(ans)
    return acc

  def run(self):
    for itr in tqdm(range(self.n)):
      for i in range(self.p):
        fit = self.fitness(self.X[i])
        if fit > self.fitness_value[i]:
          self.pbest[i] = self.X[i]
          self.fitness_value[i] = fit

        if fit > self.maxg:
          self.gbest = self.X[i]
          self.maxg = fit

        if fit == self.maxg and (self.count(self.X[i]) < self.count(self.gbest)):
          self.gbest = self.X[i]

      r1 = random.uniform(0,1)
      r2 = random.uniform(0,1)
      for p in range(self.p):
        for q in range(self.d):
          self.V[p][q] = self.w * self.V[p][q] + self.c1*r1*(self.pbest[p][q] - self.X[p][q]) + self.c2*r2*(self.gbest[q] - self.X[p][q])

          if self.V[p][q] > self.ub:
            self.V[p][q] = self.ub
          if self.V[p][q] < self.lb:
            self.V[p][q] = self.ub

          if self.sigmoid(self.V[p][q]) > random.uniform(0,1):
            self.X[p][q] = 1
          else:
            self.X[p][q] = 0


In [37]:
def get(p, lb, ub, it, c1, c2, w, node1, node2, lr, epochs, df):
  x = df.iloc[:,:-1].apply(func= normalize, axis=0).values
  y = df.iloc[:,-1].values.astype(int)

  sep = (int)(x.shape[0] * 0.1)
  x_train = x[:-sep, :]
  y_train = y[:-sep]

  x_test = x[sep:, :]
  y_test = y[sep:]

  pp = PSO(p,x_train.shape[1],lb,ub,it,c1,c2,w, x_train, y_train,node1,node2,lr,epochs)

  ip = []
  ip_test = []
  op = y_train
  for i in range(len(pp.gbest)):
    if pp.gbest[i] == 1:
      ip.append(x_train[:,i])
      ip_test.append(x_test[:,i])
  
  ip = np.array(ip).reshape((x_train.shape[0],-1))
  ip_test = np.array(ip_test).reshape((x_test.shape[0],-1))

  #neural network
  net1 = network(node1,node2, x_train.shape[1], lr)
  net1.fit(x_train, y_train, epochs)
  prediction1 = net1.predict(x_test)#no feature

  net2 = network(node1,node2, ip.shape[1], lr)
  net2.fit(ip, y_train, epochs)
  prediction2 = net2.predict(ip_test)#feature selection
  #------------------------------------------------

  print(f"number of features selected {sum(pp.gbest)}")
  print(f"Feature selection accuracy->{100*(((prediction2==y_test).sum()*1.0)/len(y_test))}")
  print(f"No Feature selection accuracy->{100*(((prediction1==y_test).sum()*1.0)/len(y_test))}")

In [38]:
p = 50 #number of particles
lb = 0 #min value after normalization
ub = 1 #max value after normalization
it = 30 #number of itrations
c1 = 1.5 # coefficient 1
c2 = 1.7 # coefficient 3
w = 0.7 #intertia

node1 = 16 #total number of nodes in layer1
node2 = 8 #total number of nodes in layer2
lr = 0.001 #learning rate
epochs = 300 #epochs in ANN
get(p,lb,ub,it,c1,c2,w,node1,node2,lr,epochs,df_pc1)


number of features selected 10
Feature selection accuracy->100.0
No Feature selection accuracy->99.69969969969969


In [39]:
get(p,lb,ub,it,c1,c2,w,node1,node2,lr,epochs,df_kc1)


number of features selected 12
Feature selection accuracy->93.8388625592417
No Feature selection accuracy->93.68088467614534


In [40]:
get(p,lb,ub,it,c1,c2,w,node1,node2,lr,epochs,df_kc2)


number of features selected 12
Feature selection accuracy->78.51063829787233
No Feature selection accuracy->81.91489361702128


In [56]:
get(p,lb,ub,it,c1,c2,w,node1,node2,lr,epochs,df_jm1)


number of features selected 17
Feature selection accuracy->89.62418300653596
No Feature selection accuracy->89.28717320261438
